In [12]:
import os 
import numpy as np 
from Recording import Recording
import settings as s

import os
import umap
import scipy.io as sio
import numpy as np
import h5py

import matplotlib
from rich import print
from rich.progress import track
from rich.traceback import install 
import matplotlib.pyplot as plt
import pandas as pd 
import seaborn as sns

from sklearn import svm
from sklearn.model_selection import train_test_split, KFold, cross_val_score, LeaveOneOut, RepeatedKFold, cross_val_predict
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
from sklearn.metrics import confusion_matrix
from sklearn.utils import shuffle

params = s.params()



# Keep in mind that it is here
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Load electrophy recordings
recs = []
main_folder = '/home/pouple/PhD/Data/To_analyze'
for folder in os.listdir(main_folder):
    cp = os.path.join(main_folder, folder)
    print('Analyzing {} ...'.format(folder))
    rec = Recording(cp, os.path.join(cp, 'SoundInfo.mat'), name=folder)
    rec.select_data_quality(quality='good')
    rec.ttl_alignment(multi=False)
    recs.append(rec)

rec = np.sum(recs)

Analyzing 09_11_2021_1 ...

Analyzing 23_11_2021_2 ...

Analyzing 08_11_2021_2 ...

Analyzing 08_11_2021_1 ...

Analyzing 09_11_2021_2 ...

Analyzing 23_11_2021_1 ...

In [3]:
def compute_svm(X, y):

    scores = []
    X, y = shuffle(X, y)
    clf = svm.SVC(kernel='linear')
    scores = cross_val_score(clf, X, y, cv=5)


    ########## Need to make sure that data is shuffled
    #scores = cross_val_score(clf, X, y, cv=5)
    return scores

In [4]:
def svm_preformance(rec):
    for i, t in enumerate([params.task1, params.task2, params.task3, params.task4]):
        scores = []
        for p in track(np.arange(0, 1000, 50), description='Compute SVM for each task ...'):
            pop_vectors = rec.complete_vectors(0, p)

            X = np.array([pop_vectors[stim][p] for stim in t for p in pop_vectors[stim]])
            if i < 2:
                y = np.array([0 if i < 8 else 1 for i, stim in enumerate(t) for p in pop_vectors[stim]])
            elif i == 2:
                y = params.y_task3
            elif i == 3:
                y = params.y_task4

            score = compute_svm(X, y)
            scores.append([np.mean(score), np.std(score)])

        scores = np.array(scores).reshape(-1, 2)

        plt.errorbar(np.arange(0, 1000, 50), scores[:, 0], label='Task {}'.format(i + 1))

    plt.legend()
    plt.savefig('performance_svm_population.png')
    plt.show()

In [5]:
def psychocurve1(rec, pb=0, pa=1000, timebin=10, vec='complete', name='test'):

    colors = ['blue']
    for tidx, t in enumerate([params.task1]):
        
        if vec == 'complete': pop_vectors = rec.get_complete_vectors(pb, pa, timebin=timebin)
        if vec == 'time': pop_vectors = rec.get_timings_vectors(pb, pa, timebin=timebin)
        if vec == 'pop': pop_vectors = rec.get_population_vectors(pb, pa)


        X = np.array([pop_vectors[stim][p] for stim in t for p in pop_vectors[stim]])
        y = np.array([0 if i < 8 else 1 for i, stim in enumerate(t) for p in pop_vectors[stim]])
        
        true_classes = np.array([i for i, stim in enumerate(t) for p in pop_vectors[stim]]) + 1

        X, y, true_classes = shuffle(X, y, true_classes)
        psycos = []
        for train_index, test_index in track(RepeatedKFold(n_repeats=3).split(X), total=5*3):
            clf = svm.SVC(kernel='linear')
            X_train, X_test = X[train_index], X[test_index]
            y_train, y_test = y[train_index], y[test_index]
            tc_train, tc_test = true_classes[train_index], true_classes[test_index]

            clf.fit(X_train, y_train)
            y_pred = clf.predict(X_test)
            correct = np.logical_not(np.logical_xor(y_pred, y_test))
            counting_vec = list(correct * tc_test)

            for i in np.unique(true_classes):
                try:
                    if i < 9:
                        psycos.append(counting_vec.count(i)/list(tc_test).count(i))
                    else:
                        psycos.append(1 - counting_vec.count(i)/list(tc_test).count(i))
                except ZeroDivisionError:
                    psycos.append(0)
    
        
        psycos = np.array(psycos).reshape(-1, 16)
        psycos = np.mean(psycos, axis=0)
        
        return psycos
"""
        plt.plot(np.geomspace(6e3, 16e3, 16), psycos, color=colors[tidx], linewidth=2, markersize=6, marker='o')
        plt.xscale('log')
        plt.legend()
        plt.savefig('p1_{}.svg'.format(name))
        

        plt.show()"""

"\n        plt.plot(np.geomspace(6e3, 16e3, 16), psycos, color=colors[tidx], linewidth=2, markersize=6, marker='o')\n        plt.xscale('log')\n        plt.legend()\n        plt.savefig('p1_{}.svg'.format(name))\n        \n\n        plt.show()"

In [6]:
def psychocurve2(rec, pb=0, pa=1000, timebin=10, vec='complete', name='test'):
    
    
    colors = ['forestgreen']
    for tidx, t in enumerate([params.task2]):
        
        if vec == 'complete': pop_vectors = rec.get_complete_vectors(pb, pa, timebin=timebin)
        if vec == 'time': pop_vectors = rec.get_timings_vectors(pb, pa, timebin=timebin)
        if vec == 'pop': pop_vectors = rec.get_population_vectors(pb, pa)


        X = np.array([pop_vectors[stim][p] for stim in t for p in pop_vectors[stim]])
        y = np.array([0 if i < 8 else 1 for i, stim in enumerate(t) for p in pop_vectors[stim]])

        true_classes = np.array([i for i, stim in enumerate(t) for p in pop_vectors[stim]]) + 1
        
        X, y, true_classes = shuffle(X, y, true_classes)
        psycos = []
        for train_index, test_index in track(RepeatedKFold(n_repeats=3).split(X), total=5*3):
            clf = svm.SVC(kernel='linear')
            X_train, X_test = X[train_index], X[test_index]
            y_train, y_test = y[train_index], y[test_index]
            tc_train, tc_test = true_classes[train_index], true_classes[test_index]

            clf.fit(X_train, y_train)
            y_pred = clf.predict(X_test)
            correct = np.logical_not(np.logical_xor(y_pred, y_test))
            counting_vec = list(correct * tc_test)

            for i in np.unique(true_classes):
                try:
                    if i < 9:
                        psycos.append(counting_vec.count(i)/list(tc_test).count(i))
                    else:
                        psycos.append(1 - counting_vec.count(i)/list(tc_test).count(i))
                except ZeroDivisionError:
                    psycos.append(0)
    
        
        psycos = np.array(psycos).reshape(-1, 16)
        psycos = np.mean(psycos, axis=0)
        
        return psycos
"""
        plt.plot(np.geomspace(20, 200, 16), psycos, color=colors[tidx], linewidth=2, markersize=6, marker='o')
        plt.xscale('log')
        plt.savefig('p2_{}.svg'.format(name))

        plt.show()"""

"\n        plt.plot(np.geomspace(20, 200, 16), psycos, color=colors[tidx], linewidth=2, markersize=6, marker='o')\n        plt.xscale('log')\n        plt.savefig('p2_{}.svg'.format(name))\n\n        plt.show()"

In [7]:
def psychocurve3(rec, pb=0, pa=1000, timebin=10, vec='complete', name='test'):
    
    task31 = [8, 12 , 16, 20, 0, 4]
    task32 = [x + 1 for x in task31]
    task33 = [x + 1 for x in task32]
    task34 = [x + 1 for x in task33]
    
    colors = ['#faa307', '#f48c06', '#e85d04', '#dc2f02']
    legends = ['45dB', '50dB', '55dB', '60dB']
    for tidx, t in enumerate([task31, task32, task33, task34]):
        
        if vec == 'complete': pop_vectors = rec.get_complete_vectors(pb, pa, timebin=timebin)
        if vec == 'time': pop_vectors = rec.get_timings_vectors(pb, pa, timebin=timebin)
        if vec == 'pop': pop_vectors = rec.get_population_vectors(pb, pa)


        X = np.array([pop_vectors[stim][p] for stim in t for p in pop_vectors[stim]])
        y = np.array([0 if i < 3 else 1 for i, stim in enumerate(t) for p in pop_vectors[stim]])

        true_classes = np.array([i for i, stim in enumerate(t) for p in pop_vectors[stim]]) + 1

        X, y, true_classes = shuffle(X, y, true_classes)
        psycos = []
        for train_index, test_index in track(RepeatedKFold(n_repeats=3).split(X), total=5*3):
            clf = svm.SVC(kernel='linear')
            X_train, X_test = X[train_index], X[test_index]
            y_train, y_test = y[train_index], y[test_index]
            tc_train, tc_test = true_classes[train_index], true_classes[test_index]

            clf.fit(X_train, y_train)
            y_pred = clf.predict(X_test)
            correct = np.logical_not(np.logical_xor(y_pred, y_test))
            counting_vec = list(correct * tc_test)

            for i in np.unique(true_classes):
                try:
                    if i < 4:
                        psycos.append(counting_vec.count(i)/list(tc_test).count(i))
                    else:
                        psycos.append(1 - counting_vec.count(i)/list(tc_test).count(i))
                except ZeroDivisionError:
                    psycos.append(0)
    
        psycos = np.array(psycos).reshape(-1, 6)
        psycos = np.mean(psycos, axis=0)
        
        return psycos

        plt.plot(np.geomspace(20, 200, 6), psycos, color=colors[tidx], label=legends[tidx], linewidth=2, markersize=6, marker='o')
        plt.xscale('log')
        plt.legend()
    plt.savefig('p3_{}.svg'.format(name))
    plt.show()
        

In [8]:
def psychocurve4(rec, pb=0, pa=1000, timebin=10, vec='complete', name='test'):
    colors = ['purple']
    for tidx, t in enumerate([params.task4]):
        
        if vec == 'complete': pop_vectors = rec.get_complete_vectors(pb, pa, timebin=timebin)
        if vec == 'time': pop_vectors = rec.get_timings_vectors(pb, pa, timebin=timebin)
        if vec == 'pop': pop_vectors = rec.get_population_vectors(pb, pa)


        X = np.array([pop_vectors[stim][p] for stim in t for p in pop_vectors[stim]])
        y = np.array([1 if i == 0 else 0 for i, stim in enumerate(t) for p in pop_vectors[stim]])

        true_classes = np.array([i for i, stim in enumerate(t) for p in pop_vectors[stim]]) + 1
        

        X, y, true_classes = shuffle(X, y, true_classes)
        psycos = []
        for train_index, test_index in track(LeaveOneOut().split(X), total=X.shape[0]):
            clf = svm.SVC(kernel='linear')
            X_train, X_test = X[train_index], X[test_index]
            y_train, y_test = y[train_index], y[test_index]
            tc_train, tc_test = true_classes[train_index], true_classes[test_index]

            clf.fit(X_train, y_train)
            y_pred = clf.predict(X_test)
            correct = np.logical_not(np.logical_xor(y_pred, y_test))
            counting_vec = list(correct * tc_test)

            for i in np.unique(true_classes):
                try:
                    if i > 1:
                        psycos.append(counting_vec.count(i)/list(tc_test).count(i))
                    else:
                        psycos.append(1 - counting_vec.count(i)/list(tc_test).count(i))
                except ZeroDivisionError:
                    psycos.append(0)
        
        psycos = np.array(psycos).reshape(-1, 6)
        psycos = np.mean(psycos, axis=0)

        plt.plot([0, 2, 4, 6, 8, 10], psycos, color=colors[tidx], linewidth=2, markersize=6, marker='o')
        plt.savefig('p4_{}.svg'.format(name))
        plt.show()
        

In [ ]:
from sklearn.metrics import mean_squared_error as mse

# Define parameters for hypersearch
parameters = {'pa': list(range(10, 1000, 10)), 
              'timebin': list(range(5, 100, 5)),
              'vec':['complete', 'pop', 'time']}

root = '/home/pouple/PhD/Data/'
task_1 = np.load(os.path.join(root, 'C/Panel_C_data.pkl'), allow_pickle=True)['mean_psykos']
task_2 = np.load(os.path.join(root, 'D/Panel_D_data.pkl'), allow_pickle=True)['mean_psykos']

plt.plot(np.geomspace(6, 16, 16), task_1)

def scoring_fit(pa, timebin, vec):
    predict_1 = psychocurve1(rec, pb=0, pa=pa,timebin=timebin, vec=vec)
    #predict_2 = psychocurve2(rec, pb=0, pa=pa,timebin=timebin, vec=vec)
    #predict_3 = psychocurve3(rec, pb=0, pa=pa,timebin=timebin, vec=vec)
    #predict_4 = psychocurve4(rec, pb=0, pa=pa,timebin=timebin, vec=vec)

    score_1 = mse(task_1, predict_1)
    return score_1

results = {}
for pa in parameters['pa']:
    for timebin in parameters['timebin']:
        for vec in parameters['vec']:
            if timebin <= pa:
                print(pa, timebin, vec)
                score = scoring_fit(pa, timebin, vec)
                results['{}_{}_{}'.format(pa, timebin, vec)] = score


    
    

10 5 complete

Output()

10 5 pop

Output()

10 5 time

...

110 45 pop

Output()

110 45 time

Output()

110 50 complete

In [ ]:
#psychocurve1(rec, pb=0, pa=1000,timebin=10, vec='pop')
#psychocurve2(rec, pb=0, pa=1000,timebin=5, vec='pop')
psychocurve3(rec, pb=0, pa=1000,timebin=5, vec='complete')
#psychocurve4(rec, pb=0, pa=1000,timebin=10, vec='complete')

In [ ]:
psychocurve1(rec, pb=0, pa=200,timebin=10, vec='complete')
psychocurve2(rec, pb=0, pa=200,timebin=10, vec='complete')
psychocurve3(rec, pb=0, pa=200,timebin=10, vec='complete')
#psychocurve4(rec, pb=0, pa=200,timebin=10, vec='complete')

In [ ]:
psychocurve1(rec, pb=0, pa=100,timebin=10, vec='complete')
psychocurve2(rec, pb=0, pa=100,timebin=10, vec='complete')
psychocurve3(rec, pb=0, pa=100,timebin=10, vec='complete')

In [ ]:
psychocurve1(rec, pb=0, pa=60,timebin=10, vec='complete')
psychocurve2(rec, pb=0, pa=60,timebin=10, vec='complete')
psychocurve3(rec, pb=0, pa=60,timebin=10, vec='complete')

In [ ]:
psychocurve1(rec, pb=0, pa=80,timebin=10, vec='complete')
psychocurve2(rec, pb=0, pa=80,timebin=10, vec='complete')
psychocurve3(rec, pb=0, pa=80,timebin=10, vec='complete')

In [ ]:
psychocurve1(rec, pb=0, pa=80,timebin=10, vec='pop')
psychocurve2(rec, pb=0, pa=80,timebin=10, vec='pop')
psychocurve3(rec, pb=0, pa=80,timebin=10, vec='pop')

In [ ]:

psychocurve1(rec, pb=0, pa=60,timebin=10, vec='pop')
psychocurve2(rec, pb=0, pa=60,timebin=10, vec='pop')
psychocurve3(rec, pb=0, pa=60,timebin=10, vec='pop')

In [ ]:
psychocurve1(rec, pb=0, pa=200,timebin=10, vec='time')
psychocurve2(rec, pb=0, pa=200,timebin=10, vec='time')
psychocurve3(rec, pb=0, pa=200,timebin=10, vec='time')

In [ ]:
psychocurve1(rec, pb=0, pa=500,timebin=100, vec='time')
psychocurve2(rec, pb=0, pa=500,timebin=100, vec='time')
psychocurve3(rec, pb=0, pa=500,timebin=100, vec='time')

In [ ]:
psychocurve1(rec, pb=0, pa=200,timebin=10, vec='complete', name='200')
psychocurve1(rec, pb=0, pa=100,timebin=10, vec='complete', name='100')
psychocurve1(rec, pb=0, pa=80,timebin=10, vec='complete', name='80')
psychocurve1(rec, pb=0, pa=60,timebin=10, vec='complete', name='60')



In [ ]:
psychocurve1(rec, pb=0, pa=1000,timebin=10, vec='complete', name='1000')


In [ ]:
psychocurve2(rec, pb=0, pa=1000,timebin=10, vec='complete', name='1000')
psychocurve2(rec, pb=0, pa=200,timebin=10, vec='complete', name='200')
psychocurve2(rec, pb=0, pa=100,timebin=10, vec='complete', name='100')
psychocurve2(rec, pb=0, pa=80,timebin=10, vec='complete', name='80')
psychocurve2(rec, pb=0, pa=60,timebin=10, vec='complete', name='60')

In [ ]:
psychocurve1(rec, pb=0, pa=200,timebin=10, vec='complete', name='time10')
psychocurve1(rec, pb=0, pa=200,timebin=20, vec='complete', name='time20')
psychocurve1(rec, pb=0, pa=200,timebin=40, vec='complete', name='time40')
psychocurve1(rec, pb=0, pa=200,timebin=100, vec='complete', name='time100')

In [ ]:
psychocurve3(rec, pb=0, pa=200,timebin=10, vec='complete', name='200')
psychocurve3(rec, pb=0, pa=100,timebin=10, vec='complete', name='100')
psychocurve3(rec, pb=0, pa=80,timebin=10, vec='complete', name='80')
psychocurve3(rec, pb=0, pa=60,timebin=10, vec='complete', name='60')